In [1]:
import random

## space availablility find

In [15]:
class Space():
    def __init__(self, height, width, num_hospitals):
        self.height = height
        self.width = width
        self.num_hospitals = num_hospitals
        self.houses = set()
        self.hospitals = set()
        
    def add_house(self, row, col):
        self.houses.add((row,col))
        
    def add_hospital(self, row, col):
        self.hospitals.add((row,col))

    def available_spaces(self):
        """ Returns all cells not currently used by a house or hospitals """
        # consider all possible cells
        candidates = set(
            (row,col)
            for row in range(self.height)
            for col in range(self.width)
        )
        # Remove all houses and hospitals
        for house in self.houses:
            candidates.remove(house)
        for hospital in self.hospitals:
            candidates.remove(hospital)
        return candidates

space = Space(height=10, width=20, num_hospitals=3)
for i in range(15):
    space.add_house(
        random.randrange(space.height), 
        random.randrange(space.width))
# print("Available spaces: ", space.available_spaces())
print("Number of Available spaces: ", len(space.available_spaces()))

Number of Available spaces:  187


## Cost count part

In [1]:
class Space():
    def __init__(self):
        self.houses = set()
        
    def add_house(self, row, col):
        self.houses.add((row,col))


    def get_cost(self, hospitals):
        """ calculate sum of distances from houses to nearest hospital. """
        cost = 0
        for house in self.houses:
            cost += min(
                abs(house[0]- hospital[0]) + abs(house[1] - hospital[1])
                for hospital in hospitals
            )
        return cost
space = Space()

# add 3 houses
space.add_house(0,0)
space.add_house(1,3)
space.add_house(4,4)
hospitals = {(0,2), (3,3)}
print("Calculating distances: ")
for house in space.houses:
    distances = []
    for hospital in hospitals:
        d = abs(house[0] - hospital[0]) + abs(house[1] - hospital[1])
        distances.append((hospital, d))
    print(f"House {house} : ")
    for hosp, dist in distances:
        print(f" -> Hospital {hosp}: Distance = {dist}")
    min_dist = min(d[1] for d in distances)
    print(f"Nearest = {min_dist}\n")
    
total_cost = space.get_cost(hospitals)
print("total cost: ", total_cost)

Calculating distances: 
House (4, 4) : 
distances : [((0, 2), 6), ((3, 3), 2)]
 -> Hospital (0, 2): Distance = 6
 -> Hospital (3, 3): Distance = 2
Nearest = 2

House (1, 3) : 
distances : [((0, 2), 2), ((3, 3), 2)]
 -> Hospital (0, 2): Distance = 2
 -> Hospital (3, 3): Distance = 2
Nearest = 2

House (0, 0) : 
distances : [((0, 2), 2), ((3, 3), 6)]
 -> Hospital (0, 2): Distance = 2
 -> Hospital (3, 3): Distance = 6
Nearest = 2

total cost:  6


## neighbor selection part

In [19]:
class Space():
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.houses = set()
        self.hospitals = set()
        
    def add_house(self, row, col):
        self.houses.add((row, col))
        
    def add_hospital(self, row, col):
        self.hospitals.add((row, col))
        
    
    def get_neighbors(self, row, col):
        """ Returns neighbors not already containing a house or hospital """
        candidates = [
            (row - 1, col),
            (row + 1, col),
            (row, col - 1),
            (row, col + 1)
        ]
        neighbors = []
        for r, c in candidates:
            if (r,c) in self.houses or (r,c) in self.hospitals:
                continue
            if 0 <= r < self.height and 0 <=c < self.width:
                neighbors.append((r,c))
        return neighbors


space = Space(height=5, width=5)

# add a house at (1,2) and a hospital at (2,3)
space.add_house(1,2)
space.add_hospital(2,3)

# now get neighbors of (2,2)
neighbors = space.get_neighbors(2,2)

print("Valid neighbors of (2,2): ", neighbors)

Valid neighbors of (2,2):  [(3, 2), (2, 1)]


## image set

In [46]:
class Space():
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.houses = set()
        self.hospitals = set()
        
    def add_house(self, row, col):
        self.houses.add((row, col))
        
    def add_hospital(self, row, col):
        self.hospitals.add((row, col))
        
    
    def get_neighbors(self, row, col):
        """ Returns neighbors not already containing a house or hospital """
        candidates = [
            (row - 1, col),
            (row + 1, col),
            (row, col - 1),
            (row, col + 1)
        ]
        neighbors = []
        for r, c in candidates:
            if (r,c) in self.houses or (r,c) in self.hospitals:
                continue
            if 0 <= r < self.height and 0 <=c < self.width:
                neighbors.append((r,c))
        return neighbors
    def output_image(self, filename):
        """ generates image with all houses and hospitals """
        from PIL import Image, ImageDraw, ImageFront
        cell_size = 100
        cell_border = 2
        cost_size = 40
        padding = 10
        # create a blank canvas
        img = Image.new(
            "RGBA",
            (self.width * cell_size,
            self.height * cell_size + cost_size + padding * 2),
            "white"
        )
        house = Image.open("assets/images/house.png").resize(
            cell_size, cell_size
        )
        hospital = Image.open("assets/images/hospital.png").resize(
            cell_size, cell_size
        )
        font = ImageFont.truetype("assets/fonts/OpenSans-Regular.ttf", 30)
        draw = ImageDraw.Draw(img)
        for i in range(self.height):
            for j in range(self.width):
                rect = [
                    (j * cell_size + cell_border,
                    i * cell_size + cell_border),
                    ((j+1) * cell_size - cell_border,
                    (i+1) * cell_size - cell_border)
                ]
                draw.rectangle(rect, fill="black")
                if (i,j) in self.houses:
                    img.paste(house, rect[0], house)
                if (i,j) in self.hospitals:
                    img.paste(hospital, rect[0], hospital)
        # Add cost
        draw.rectangle(
            (0, self.height * cell_size, self.width * cell_size,
             self.height * cell_size + cost_size + padding * 2),
            "black"
        )
        draw.text(
            (padding, self.height * cell_size + padding),
            f"Cost: {self.get_cost(self.hospitals)}",
            fill="white",
            font=font
        )

        img.save(filename)

# Create a new space and add houses randomly
s = Space(height=10, width=20, num_hospitals=3)
for i in range(15):
    s.add_house(random.randrange(s.height), random.randrange(s.width))

# Use local search to determine hospital placement
hospitals = s.hill_climb(image_prefix="hospitals", log=True)


## Hill climbing algorithm implement

In [44]:
import random
class Grid:
    def __init__(self, height, width, houses, num_hospitals=1):
        self.height = height
        self.width = width
        self.houses = set(houses)
        self.num_hospitals = num_hospitals
        self.hospitals = set()
    def available_spaces(self):
        return {
            (r,c)
            for r in range(self.height)
            for c in range(self.width)
            if (r,c) not in self.houses and (r, c) not in self.hospitals
        }
    def get_neighbors(self, row, col):
        candidates = [
             (row-1, col), (row+1, col), (row, col-1), (row, col+1)
        ]
        return [
            (r,c)
            for r,c in candidates
            if 0 <= r <self.height and 0 <= c <self.width and (r, c) not in self.houses
        ]
    def get_cost(self, hospitals):
        cost = 0
        for house in self.houses:
            cost += min(
                abs(house[0] - h[0]) + abs(house[1] - h[1])
                for h in hospitals
            )
        return cost
    def hill_climb(self, max_steps = 10):
        count = 0
        self.hospitals = set()
        
        for _ in range(self.num_hospitals):
            self.hospitals.add(random.choice(list(self.available_spaces())))
        print(f"initial hospital: {self.hospitals}")
        print(f"initial cost: {self.get_cost(self.hospitals)}\n")

        while count < max_steps:
            count += 1
            best_neighbors = []
            best_cost = None
            
            for hospital in self.hospitals:
                for replecement in self.get_neighbors(*hospital):
                    neighbor = self.hospitals.copy()
                    neighbor.remove(hospital)
                    neighbor.add(replecement)
                    cost = self.get_cost(neighbor)

                    if best_cost is None or cost < best_cost:
                        best_cost = cost
                        best_neighbors = [neighbor]
                    elif cost == best_cost:
                        best_neighbors.append(neighbor)
            current_cost = self.get_cost(self.hospitals)
            if best_cost >= current_cost:
                print("No better move found, stopping")
                break
            self.hospitals = random.choice(best_neighbors)
            print(f"step {count}: moved hospital -> {self.hospitals}, cost={best_cost}")
        print("\nFinal hospital positions:", self.hospitals)
        print("Final cost:", self.get_cost(self.hospitals))


grid = Grid(
    height=3,
    width=3,
    houses=[(0, 0), (2, 2)],
    num_hospitals=1
)

grid.hill_climb()

initial hospital: {(0, 2)}
initial cost: 4

No better move found, stopping

Final hospital positions: {(0, 2)}
Final cost: 4


## Full algorithm implement

In [62]:
class Space():
    def __init__(self, height, width, num_hospitals):
        """Create a new state space with given dimensions."""
        self.height = height
        self.width = width
        self.num_hospitals = num_hospitals
        self.houses = set()
        self.hospitals = set()
        
    def add_house(self, row, col):
        self.houses.add((row, col))
        
    def available_spaces(self):
        """Returns all cells not currently used by a house or hospital."""
        candidates = set(
            (row,col)
            for row in range(self.height)
            for col in range(self.width)
        )
        # remove all houses and hospitals
        for house in self.houses:
            candidates.remove(house)
        for hospital in self.hospitals:
            candidates.remove(hospital)
        return candidates
    def get_cost(self, hospitals):
        cost = 0
        for house in self.houses:
            cost +=min(
                abs(house[0] - hospital[0]) + abs(house[1] - hospital[1])
                for hospital in hospitals
            )
        return cost
        
    def get_neighbors(self, row, col):
        """ Returns neighbors not already containing a house or hospital """
        candidates = [
            (row - 1, col),
            (row + 1, col),
            (row, col - 1),
            (row, col + 1)
        ]
        neighbors = []
        for r, c in candidates:
            if (r,c) in self.houses or (r,c) in self.hospitals:
                continue
            if 0 <= r < self.height and 0 <=c < self.width:
                neighbors.append((r,c))
        return neighbors

    
    def hill_climb(self, maximum=None, image_prefix=None, log=False):
        """ Performs hill-climbing to find a solution """
        count = 0

        # add hospitals randomly
        self.hospitals = set()
        for i in range(self.num_hospitals):
            self.hospitals.add(random.choice(list(self.available_spaces())))
            
        if log:
            print("Initial state: cost", self.get_cost(self.hospitals))
        if image_prefix:
            self.output_image(f"{image_prefix}{str(count).zfill(3)}.png")
            
        # continue until we reach maximum number of iterations
        while maximum is None or count < maximum:
            count +=1
            best_neighbors = []
            best_neighbor_cost = None
            
            # consider all hospitals to move
            for hospital in self.hospitals:
                
                # consider all neighbors for that hospital
                for replacement in self.get_neighbors(*hospital):
                    
                    # generate a neighboring set of hospitals
                    neighbor = self.hospitals.copy()
                    neighbor.remove(hospital)
                    neighbor.add(replacement)
                    # check if neighbor is bet so far
                    cost = self.get_cost(neighbor)
                    if best_neighbor_cost is None or cost < best_neighbor_cost:
                        best_neighbor_cost = cost
                        best_neighbors = [neighbor]
                    elif best_neighbor_cost == cost:
                        best_neighbors.append(neighbor)
                        
            # None of the neighbors are better than the current state
            if best_neighbor_cost >= self.get_cost(self.hospitals):
                return self.hospitals
            # Move to a highest-valued neighbor
            else:
                if log:
                    print(f"found betther neighbor: cost {best_neighbor_cost}")
                self.hospitals = random.choice(best_neighbors)
            # generate image
            if image_prefix:
                self.output_image(f"{image_prefix}{str(count).zfill(3)}.png")

     
        
    def output_image(self, filename):
        """Generates image with all houses and hospitals."""
        from PIL import Image, ImageDraw, ImageFont
        cell_size = 100
        cell_border = 2
        cost_size = 40
        padding = 10

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.width * cell_size,
             self.height * cell_size + cost_size + padding * 2),
            "white"
        )
        house = Image.open("assets/images/House.png").resize(
            (cell_size, cell_size)
        )
        hospital = Image.open("assets/images/Hospital.png").resize(
            (cell_size, cell_size)
        )
        font = ImageFont.truetype("assets/fonts/OpenSans-Regular.ttf", 30)
        draw = ImageDraw.Draw(img)

        for i in range(self.height):
            for j in range(self.width):

                # Draw cell
                rect = [
                    (j * cell_size + cell_border,
                     i * cell_size + cell_border),
                    ((j + 1) * cell_size - cell_border,
                     (i + 1) * cell_size - cell_border)
                ]
                draw.rectangle(rect, fill="black")

                if (i, j) in self.houses:
                    img.paste(house, rect[0], house)
                if (i, j) in self.hospitals:
                    img.paste(hospital, rect[0], hospital)

        # Add cost
        draw.rectangle(
            (0, self.height * cell_size, self.width * cell_size,
             self.height * cell_size + cost_size + padding * 2),
            "black"
        )
        draw.text(
            (padding, self.height * cell_size + padding),
            f"Cost: {self.get_cost(self.hospitals)}",
            fill="white",
            font=font
        )

        img.save(filename)



# Create a new space and add houses randomly
s = Space(height=10, width=20, num_hospitals=3)
for i in range(15):
    s.add_house(random.randrange(s.height), random.randrange(s.width))

# Use local search to determine hospital placement
hospitals = s.hill_climb(image_prefix="hospitals", log=True)

Initial state: cost 70
found betther neighbor: cost 66
found betther neighbor: cost 64
found betther neighbor: cost 62
found betther neighbor: cost 61
found betther neighbor: cost 60
found betther neighbor: cost 59
found betther neighbor: cost 58
